<a href="https://colab.research.google.com/github/bruAristimunha/Emotion-Recognition-Transformers/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend moabb braindecode
!pip install --upgrade mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.0 MB/s 
     |████████████████████████████████| 177 kB 43.8 MB/s 
     |████████████████████████████████| 636 kB 59.5 MB/s 
     |████████████████████████████████| 7.5 MB 51.1 MB/s 
     |████████████████████████████████| 365 kB 69.5 MB/s 
     |████████████████████████████████| 242 kB 65.2 MB/s 
     |████████████████████████████████| 155 kB 64.3 MB/s 
  Created wheel for pyriemann: filename=pyriemann-0.3-py2.py3-none-any.whl size=78033 sha256=933083e003ae2c4004d62528170700af9b644c95f2f240d336e519b4b37de8b5
  Stored in directory: /root/.cache/pip/wheels/0b/1b/bf/a537f9e17e6c3490004ede419c72f863af1d0d765d25e532ef
Successfully built pyriemann
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Looking in indexes: https://pypi.org/simple, https://us-

# Structure of the Project




N distinct individuals, with train datasets ${\left\{(X_n,y_n)\right\}}^N_{n=1}$ for each. 

New individual T, with $(X_T, y_T)$ and less training examples than the other N.

We want to use the N models created for each individual to create a combination that classifies well for a new individual T.

***First step: Preprocessing**

--> Load of data from MOABBDataset

--> Preprocessing: Preprocessor from braindecode.preprocessing

*   'pick_types' - Keep EEG sensors
*   Scale - change scale from V to uV
*   'filter' - bandpass filter
*   'exponential moving standardization' - moving average
*   **Euclidean Alignment** : but, to execute this, we first need to split the data in subjects and trials (and Training/Test?)

--> Cut compute windows: from braindecode.preprocessing import create_windows_from_events

Slice the dataset in pieces (windows) of a certain $\Delta t$.

***Second step: create models for each subjects**

***Third step: create the ensemble**

***Fourth step: find good metrics for measuring the efficiency**


# The Euclidean alignment



*Reference: He H, Wu D. Transfer Learning for Brain-Computer Interfaces: A Euclidean Space Data Alignment Approach. IEEE Trans Biomed Eng. 2020 Feb;67(2):399-410. doi: [10.1109/TBME.2019.2913914](https://ieeexplore.ieee.org/document/8701679). Epub 2019 Apr 29. PMID: 31034407.*

Alignment of the data from the N different individuals in the Eucidean space to make them more similar. 

Let $X^{j}_i$ be the matrix of EEG signals for trial i$\in \left\{1, 2, ..., n\right\}$ from the subject (individual) j. We first compute the aritimetic mean of all covariance matrix

\begin{equation}
\bar{R^j}=1/n \sum^{n}_{i=1}X^j_i {X^j_i}^T
\end{equation}

Then, we perform the alignment on each trial, that is the matrix multiplication 

\begin{equation}
\tilde{X^j_i}={\bar{R^j}}^{\,-1/2} X^j_i
\end{equation}

Summarizing: For each individual j, $\left\{X^j_i\right\}^n_{i=1} -^T→ \left\{\tilde{X^j_i}\right\}^n_{i=1}$.

This new data has the mean covariance matrices of all subjects equal to the identity matrix ($I$), and, therefore, the distributions are more similar.


Just some tests:

In [ ]:
#generating synthetic eeg data
n_examples=5
n_channels=5
sfreq=128
input_size_s=1
rng = np.random.RandomState(42)
X = rng.randn(n_examples, n_channels, int(sfreq * input_size_s))
X = X.astype(np.float32)

In [ ]:
print(X)

In [ ]:
!pip install pyriemann

In [ ]:
from pyriemann.utils import mean
R=mean.mean_euclid(X[..., np.newaxis])
print((R**(-1/2)).shape)
print(X.shape)
print(X@(R**(-1/2)))


In [ ]:
# Is working! (but this way to compute the covariance woks only with 2D arrays)
import scipy
#Matrix 
A=np.array([[2,3,4],[7,2,5]])
#"Average" covariance R (is positive definite)
R=A@A.T
print(R)
#Find the matrix square root S of R (such that R=S@S)
S=scipy.linalg.sqrtm(R)
print(S.dot(S))
#Invs=S^{-1}=(R^{1/2})^{-1}=R^{-1/2}
InvS=np.linalg.inv(S)
#Alignment of A: An=R^{-1/2}@A
An=InvS@A
print(An)
#Property: An@An.T=I
print(An@An.T)

##Testing the Euclidean Alignment (EA)

We want to evaluate the performance in new subjects.
Here, we are going to use the ShallowConvNet or EEGNet networks.

(\*) The comparision is going to be made using *leave-one-out* or *k-fold*.

- **Leave-one-out (LOO)**: 
- **k-fold**:


### Download and preprocess the data

In [ ]:
#Structure of the project
from braindecode.datasets import MOABBDataset

dataset=MOABBDataset(dataset_name="BNCI2014001", subject_ids=[1,2,3,4,5])

/usr/local/lib/python3.7/dist-packages/moabb/datasets/download.py:53: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BNCI_PATH"
  set_config(key, get_config("MNE_DATA"))


MNE_DATA is not already configured. It will be set to default location in the home directory - /root/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location


100%|█████████████████████████████████████| 42.8M/42.8M [00:00<00:00, 31.0GB/s]
SHA256 hash of downloaded file: 054f02e70cf9c4ada1517e9b9864f45407939c1062c6793516585c6f511d0325
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 43.8M/43.8M [00:00<00:00, 26.1GB/s]
SHA256 hash of downloaded file: 53d415f39c3d7b0c88b894d7b08d99bcdfe855ede63831d3691af1a45607fb62
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 43.1M/43.1M [00:00<00:00, 38.1GB/s]
SHA256 hash of downloaded file: 5ddd5cb520b1692c3ba1363f48d98f58f0e46f3699ee50d749947950fc39db27
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|████████████████████████████████

48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
4

In [ ]:
#LOO - Leave-One-Out
from sklearn.model_selection import LeaveOneOut
import mne

Preprocess the data: reescale, compute moving average, filter

In [ ]:
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor, scale)
mne.set_log_level('ERROR')

low_cut_hz = 0.5  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(scale, factor=1e6, apply_on_array=True),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size),
]

# Transform the data
preprocess(dataset, preprocessors)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Cut compute windows.

Windows are the network inputs

In [ ]:
# Cut Compute Windows: another step of preprocessing
from braindecode.preprocessing import create_windows_from_events
import mne
mne.set_log_level('ERROR')

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Another preprocessing: Euclidean Alignment 

In [ ]:
import numpy as np 
from scipy.linalg import sqrtm, inv
from numpy import iscomplexobj, real, any, isfinite
### Define this as Transformation, maybe...
def euclidean_alignment(data, y=None):

  assert len(data.shape) == 3
  # r = np.matmul(data, data.transpose((0, 2, 1))).mean(0)

  #calculate covariance matrix of each trial
  # list_cov = list()
  r = 0
  for trial in data:
      cov = np.cov(trial, rowvar=True)
      r += cov

  r = r/len(data)

  if iscomplexobj(r):
      print("covariance matrix problem")
  if iscomplexobj(sqrtm(r)):
      print("covariance matrix problem sqrt")

  r_op = inv(sqrtm(r))

  if iscomplexobj(r_op):
      print("WARNING! Covariance matrix was not SPD somehow. "+
            "Can be caused by running ICA-EOG rejection, if "+
            "not, check data!!")
      r_op = real(r_op).astype(np.float64)
  elif not any(isfinite(r_op)):
      print("WARNING! Not finite values in R Matrix")

  result = np.matmul(r_op, data)
  return result


Take number of windows and channels used.

In [ ]:
Aux=windows_dataset.description
Test_Aux=Aux[Aux['subject']==1]; Test_Aux_list=Test_Aux.index.tolist()
Train_Aux=Aux[Aux['subject']!=1]; Train_Aux_list=Train_Aux.index.tolist()
Dic={"train":Train_Aux_list,"test":Test_Aux_list}; train_test=windows_dataset.split(Dic)

In [ ]:
Test_aux=train_test['test']
n_chans=Test_aux[0][0].shape[0]
print(n_chans)
input_window_samples = Test_aux[0][0][0].shape[0]
print(input_window_samples)

22
1125


**Create model using ShallowConvNet**

In [ ]:
# Create model (Using ShallowConvNet)
# Execute this after defining the train and test sets 
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = False
# Set random seed to be able to roughly reproduce results
# Note that with cudnn benchmark set to True, GPU indeterminism
# may still make results substantially different between runs.
# To obtain more consistent results at the cost of increased computation time,
# you can set `cudnn_benchmark=False` in `set_random_seeds`
# or remove `torch.backends.cudnn.benchmark = True`
seed = 20200222
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
# Extract number of chans and time steps from dataset

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
    filter_time_length=10,
)

# Send model to GPU
if cuda:
    model.cuda()

**Function for the training.**

Remember to add regularization and change parameters later.

In [ ]:
# train
# I didn't add dropout yet

from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier
from sklearn.base import clone

def train(model, train_set, val_set, device):
  # eval: cross validation evaluation
  # Good hyperparameters values for shallow net
  lr = 0.0625 * 0.01
  weight_decay = 0

  batch_size = 16
  n_epochs = 2

  # Model (Classifier)
  clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(val_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", 
                      LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
  )

  clf=clone(clf)

  # Model training for a specified number of epochs. `y` is None as it is already supplied
  # in the dataset.
  clf.fit(train_set, y=None, epochs=n_epochs)
  return clf

### Auxiliary functions

In [ ]:
import pandas

In [ ]:
# FUNCTION TO SPLIT IN TRAIN AND VALIDATION
def Split_Train_Val(Aux_train_val, val_subj=None):
  """
  Aux_train_val: DataFrame
  subj: int
  """

  if val_subj!=None:
    Val_id=Aux_train_val[Aux_train_val['subject']==val_subj]
    Val_id_list=Val_id.index.tolist()
    Train_id=Aux_train_val[Aux_train_val['subject']!=val_subj]
    Train_id_list=Train_id.index.tolist()
    
  else:
    Aux_train=Aux_train_val[Aux_train_val['session']=='session_T']
    Aux_train_list=Aux_train.index.tolist()
    Aux_val=Aux_train_val[Aux_train_val['session']=='session_E']
    Aux_val_list=Aux_val.index.tolist()

  Dic={"train":Aux_train_list,"val": Aux_val_list}
  train_val=windows_dataset.split(Dic)
  Train=train_val['train']; Val=train_val['val']
  return Train, Val

In [ ]:
from scipy.linalg.special_matrices import dft
def Split_dataset(DF):
  list_=[]
  idx=[1,2,3,4,5]

  for i in idx:
    Aux=DF[DF['subject']==i]
    list_.append(Aux.index.tolist())

  Dic={f'{idx[i]}':list_[i] for i in range(5)}
  Data_subj=windows_dataset.split(Dic)

  return Data_subj

In [ ]:
def EA_dataset(Data_subj):
  idx=[1,2,3,4,5]
  for i in idx:
    Data_subj[f'{i}']=preprocess(Data_subj[f'{i}'],
                                 [Preprocessor(euclidean_alignment,apply_on_array=True)])
  return Data_subj

### Split the data

In [ ]:
  DF=windows_dataset.description

###**Part 1** 

One network is trained with almost the entire dataset (minus one individual, the one we want to test).

** We align all subjects together and then we chose wich one is the new (test) or we need to align each in their own space? I think it's the last one, because we want all individuals with similar distributions, and therefore, each one with same mean covariance.

- Without EA
- With EA

Then, we compare (*) the difference between: (i) train accuracy and loss; (ii) the predictions in the new individual (maby we can compute the train accuracy to see if the difference in fitting with aligned and non aligned data - is it harder?).

OLD CODE

NEW CODE

In [ ]:
from sklearn.base import clone
from skorch.helper import SliceDataset

In [ ]:
# Train set 1 = Splitted Dataset (without EA) 
# Train set is every individual except the last one

def Part1_loo(model, Data_subjects, device, val_subj=None):

  loo=LeaveOneOut()
  list_s= list(range(5))

  models_list = []
  predicts_list = []
  # Using Leave-One-Out validation
  for train_idx, test_idx in loo.split(list_s):
    train_idx=train_idx+1; test_idx=test_idx+1
    print("Test subject:",test_idx[0])

    # Split in Train and Test
    Test_subj=Data_subjects[f'{test_idx[0]}']
    Train_Aux=pandas.concat([Data_subjects[f'{i}'].description for i in train_idx])    

    # Split in Train and Validation
    Train, Val = Split_Train_Val(Train_Aux, val_subj=val_subj)

    clf=train(model, Train, Val, device)

    y_pred = clf.predict(Test_subj)
    y_true = list(SliceDataset(Test_subj, 1))

    models_list.append(clf)
    predicts_list.append((y_pred,y_true))
    
  return models_list, predicts_list


In [ ]:
# Dataset splitted in subjects
Data_subjects=Split_dataset(DF)
# Dataset splitted in subjects with Euclidean Alignment
Data_subjects_EA=EA_dataset(Data_subjects)
 
models_list, predicts_list = Part1_loo(model, Data_subjects, 
                                       device, val_subj=None)

Test subject: 1
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3368        1.1749            0.2743        2.3165  0.0006  7.0004
      2            0.9757        0.4609            0.5729        0.8600  0.0005  7.0731
      3            0.9931        0.2712            0.6389        0.7657  0.0003  7.2603
      4            1.0000        0.2109            0.6354        0.7484  0.0001  7.1994
      5            1.0000        0.2092            0.6389        0.7504  0.0000  7.1879
Test subject: 2
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3368        1.2206            0.2882        2.7488  0.0006  7.2750
      2            0.9826        0.4556            0.5972        0.8337  0.0005  7.1715


In [ ]:
import numpy as np

np.savez(pathsave, models_list=models_list, Train=Train, Val=Val)

2

In [ ]:
import numpy as np
for idx, test_subj in enumerate(predicts_list):
  filename = f"loo_part_1_without_EA_subject_{idx}.npz"
  np.savez(file=filename, y_pred = test_subj[0], y_true = test_subj[1])

In [ ]:
np_struct = np.load("loo_part_1_without_EA_subject_0.npz")

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from glob import glob
file_list = glob("*without*.npz")
file_list.sort()
without_ea = []

for filename in file_list:
  np_struct = np.load(filename)
  bac = balanced_accuracy_score(y_true=np_struct['y_true'], y_pred=np_struct['y_pred'])
  print(bac)
  without_ea.append(bac)

0.8194444444444445
0.265625
0.453125
0.3211805555555555
0.27777777777777773


In [ ]:
import pandas as pd
metrica_ba = pd.DataFrame([without_ea, with_ea]).T
metrica_ba.columns = ['sem_ea', 'com_ea']

In [ ]:
metrica_ba

,sem_ea,com_ea
0,0.819444,0.798611
1,0.265625,0.272569
2,0.453125,0.440972
3,0.321181,0.331597
4,0.277778,0.284722


In [ ]:
np.mean(without_ea)

0.42743055555555565

In [ ]:
np.mean(with_ea)

0.4256944444444445

In [ ]:
models_list_EU, predicts_list_EU = Part1_loo(model, Data_subjects_EA, 
                                       device, val_subj=None)

Test subject: 1
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.9792        0.8450            0.6007        0.8160  0.0006  2.1739
      2            0.9861        0.2892            0.6111        0.8096  0.0000  2.0411
Test subject: 2
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.9722        0.8628            0.5938        0.8343  0.0006  2.0054
      2            0.9757        0.3153            0.5938        0.8241  0.0000  1.9909
Test subject: 3
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.9792        0.8828            0.6042        0.7687 

In [ ]:
import numpy as np
for idx, test_subj in enumerate(predicts_list_EU):
  filename = f"loo_part_1_with_EA_subject_{idx}.npz"
  np.savez(file=filename, y_pred = test_subj[0], y_true = test_subj[1])

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from glob import glob
file_list = glob("*with_EA*.npz")
file_list.sort()
with_ea=[]
for filename in file_list:
  np_struct = np.load(filename)
  bac = balanced_accuracy_score(y_true=np_struct['y_true'], y_pred=np_struct['y_pred'])
  with_ea.append(bac)
  print(bac)

0.7986111111111112
0.2725694444444444
0.4409722222222222
0.3315972222222222
0.2847222222222222


###**Part 2**

N-1 networks, each trained with data from a single individual.

- Without EA
- With EA 

Then, we compare (\*) the difference between: (i) train accuracy and loss of each network; (ii) the predictions in the new individual  of **each network** (maby we can compute the train accuracy to see if the difference in fitting with aligned and non aligned data - is it harder?).


NEW CODE

In [ ]:
# Train set 1 = Splitted Dataset (without EA) 
# Train set is every individual except the last one

def Part2_loo(model, Data_subjects, device, val_subj=None):

  DF=windows_dataset.description
  loo=LeaveOneOut()
  list_s= list(range(5))

  model_loo=[]

  # Using Leave-One-Out validation
  for train_idx, test_idx in loo.split(list_s):
    train_idx=train_idx+1; test_idx=test_idx+1
    print("Test subject:",test_idx[0])

    # Split in Train and Test
    Test_subj=Data_subjects[f'{test_idx[0]}']
    Train_Aux=pandas.concat([Data_subjects[f'{i}'].description for i in train_idx])   
    models_list = []   

    for id_person in train_idx:
      print("   Model for train subject", id_person)
      Train_subj=Train_Aux[Train_Aux['subject']==id_person]
      # Split in Train and Validation
      Train, Val = Split_Train_Val(Train_subj, val_subj=val_subj)

      clf=train(model, Train, Val, device)
      models_list.append(clf)

    model_loo.append(models_list)


In [ ]:
Part2_loo(model, Data_subjects, device,val_subj=None)

Test subject: 1
   Model for train subject 2
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.2535        1.6258            0.2535        3.8704  0.0006  1.7582
      2            0.4132        1.1744            0.3299        3.0085  0.0006  1.7130
      3            0.2847        0.9085            0.2604        2.9103  0.0006  1.7352
      4            0.2743        0.6731            0.2604        2.7534  0.0006  1.7530
      5            0.4792        0.6010            0.3056        1.7063  0.0005  1.7744
      6            0.8090        0.4521            0.4514        1.1874  0.0004  1.7933
      7            0.9722        0.4405            0.5486        0.9641  0.0004  1.7745
      8            0.9826        0.3724            0.5625        0.8839  0.0003  1.7783
      9            0.9931        0.3308            0.5729        0.8527  0.

In [ ]:
Part2_loo(model, Data_subjects_EA, device,val_subj=None)

Test subject: 1
   Model for train subject 2
  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3194        1.6145            0.2778        3.7209  0.0006  1.6918
      2            0.2500        1.2270            0.2535        4.3514  0.0006  1.6542
      3            0.2639        0.9100            0.2639        2.8348  0.0006  1.6682
      4            0.6181        0.7151            0.4062        1.5724  0.0006  1.6843
      5            0.7118        0.6003            0.4514        1.2557  0.0005  1.6976
      6            0.8646        0.4343            0.5312        1.0523  0.0004  1.7023
      7            0.9444        0.3895            0.5903        0.9124  0.0004  1.7342
      8            0.9792        0.3460            0.6076        0.8450  0.0003  1.7359
      9            0.9826        0.3503            0.5972        0.8163  0.

###**Part 3 (?)**

Maby we can evaluate the EA in this case too, but I don't know if is really relevant, because it's just a combination of the N-1 networks. If a certain network has a determined difference between predictions with and without EA, then in the ensemble, the percentual difference of this network with and without EA will be the same. Another problem is that, to create the ensemble we need to use the data from the new individual. So, the comparision with parts 1 and 2 couldn't be the same.

If it's relevant, we create an ensemble with the N-1 models trained with one indivudial each, and then compare the predictions with and without the alignment. 

The ensemble weights can be determined based on different methods: metrics and meta-models. So, this part needs to be done after the implementation of the ensembles.

In [ ]:
#